In [1]:
import pandas as pd
import numpy as np
import os

## Simple EDA


In [2]:
data_path = os.path.join(os.getcwd(), "jeju_data_ver1")
filename = "201901-202003.csv"

df = pd.read_csv(os.path.join(data_path, filename))

In [3]:
df.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [7]:
date_dict = df["REG_YYMM"].unique()
sido_dict = df["CARD_SIDO_NM"].unique()
ccg_dict = df["CARD_CCG_NM"].unique()
store_dict = df["STD_CLSS_NM"].unique()

In [39]:
upjong = set()
for i in store_dict:
    if i.endswith("업"):
        upjong.add(i.split()[-1])
    else:
        upjong.add(i)
upjong

{'기타음식료품위주종합소매업',
 '대행업',
 '마사지업',
 '면세점',
 '서비스업',
 '소매업',
 '슈퍼마켓',
 '여관업',
 '여행사업',
 '욕탕업',
 '운송업',
 '운영업',
 '음료점업',
 '음식점업',
 '임대업',
 '주점업',
 '체인화 편의점',
 '충전업',
 '호텔업'}

## 전처리 Scheme
1. 주소의 경우 kakao maps api를 통해 위도, 경도로 바꾼다.
2. 업종의 경우 factorize 시킨 one-hot으로 간다?

In [16]:
kdapi_token_path = "/Users/kakao/kakao_project/credentials/KakaoAK"
with open(kdapi_token_path, "r", encoding = "utf-8") as f:
    AK = f.readline().strip()

In [25]:
import requests
address = "제주시 아라동"
response = requests.get(
    url = "https://dapi.kakao.com/v2/local/search/address.json",
    params = {"query": address},
    headers = {"Authorization": "KakaoAK {}".format(AK)})


In [28]:
response.json()["documents"][0]["y"]

'33.4763364525424'